In [1]:
import numpy as np
import matplotlib.pyplot as plt

def get_snr_threshold(thres=0.1, f_name="white_noise_dnn_gain_2", num=3):
    extfs = [2, 4, 8, 16, 32, 64]
    snr_thres = np.zeros([len(extfs), num])
    snr_thres[:, :] = 100
    all_extf_bers = []
    for extf_idx in range(len(extfs)):
        extf = extfs[extf_idx]
        with open("/liymdata/liym/BLong/output/" + f_name + ".log", "r") as f:
            lines = f.readlines()
            snrs = []
            bcrs = []
            for line in lines:
                s = line.split(",")
                s_extf = int(s[0][s[0].find("=")+1:])
                if s_extf != extf:
                    continue 
                snr = int(s[1][s[1].find("=")+1:])
                bcr = np.zeros(num, dtype=np.float32)
                for i in range(num):
                    bcr[i] = float(s[2+i][s[2+i].find("=")+1:])
                snrs.append(snr)
                bcrs.append(bcr)
            snrs = np.array(snrs)
            bcrs = np.array(bcrs)
            idx = np.argsort(snrs)
            snrs = snrs[idx]
            bcrs = 1 - bcrs[idx, :]
            for i in range(len(snrs)-1):
                for j in range(num):
                    if snr_thres[extf_idx, j] == 100 and (bcrs[i, j] >= thres and bcrs[i+1, j] <= thres):
                        diff = bcrs[i+1, j] - bcrs[i, j]
                        slope = diff / (snrs[i+1] - snrs[i])
                        snr_diff = (thres - bcrs[i, j]) / slope
                        snr_thres[extf_idx, j] = snrs[i] + snr_diff
        all_extf_bers.append([snrs, bcrs])
    print(snr_thres)   
    output_data = np.hstack([np.array(extfs).reshape([-1, 1]), snr_thres])
    np.savetxt("/liymdata/liym/BLong/output/snr_threshold/ber_threshold.csv", output_data, delimiter=",")
    return snr_thres
    

def get_pdr_snr_threshold(thres=0.9, f_name="filter_packet_detection"):
    extfs = [2, 4, 8, 16, 32, 64]
    snr_thres = np.zeros([len(extfs), 3])
    for extfi in range(6):
        extf = extfs[extfi]
        data = np.loadtxt("/liymdata/liym/BLong/output/packet_detection/" + f_name + "_{}.csv".format(extf), delimiter=",")
        snrs = data[:, 1] 
        pdr = data[:, 5:]
        for j in range(3):
            for i in range(len(snrs)-1):
                if pdr[i, j] < thres and pdr[i+1, j] > thres:
                    snr_thres[extfi, j] = snrs[i] + (thres - pdr[i, j]) / (pdr[i+1, j] - pdr[i, j]) 
                    break
    output_data = np.hstack([np.array(extfs).reshape([-1, 1]), snr_thres])
    print(output_data)
    np.savetxt("/liymdata/liym/BLong/output/snr_threshold/filter_preamble_detection_rate_threshold.csv", output_data, delimiter=",")
    return snr_thres
    
ber_t = get_snr_threshold()
pdr_t = get_pdr_snr_threshold()

[[ -7.62780671  -6.91893286  -6.83532395]
 [ -9.74678483  -8.12187111  -8.55678763]
 [-12.26829855 -10.56885792 -10.36307361]
 [-14.75230184 -12.71116061 -12.13596793]
 [-17.13014538 -14.58950612 -13.75936586]
 [-18.71286314 -16.39371709 -15.46287749]]
[[  2.           2.67142857   0.17377049  -5.14054054]
 [  4.           1.17777778  -1.86428571  -5.69818182]
 [  8.          -2.48791209  -5.36271186  -9.77142857]
 [ 16.          -4.36481481  -8.22626263 -11.57108434]
 [ 32.          -6.4        -10.36166667 -13.70126582]
 [ 64.          -7.56901408 -11.44810127 -14.992     ]]


In [2]:
b = -ber_t[:, :]
diff_b = b[1:, :] - b[:-1, :]
print(np.mean(diff_b,axis=0))
b = np.mean(diff_b,axis=0)
print(b[0]-b[1], b[0]-b[2])

p = -pdr_t[:, :]
diff_p = p[1:, :] - p[:-1, :]
print(np.mean(diff_p,axis=0))

[2.21701129 1.89495685 1.72551071]
0.3220544379361512 0.4915005778707504
[2.04808853 2.32437435 1.97029189]
